In [55]:
import concurrent.futures
import time
import sys
from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler
import pandas as pd
import numpy as np
import seaborn as sns
from distance_matrix_creator import *
import matplotlib.pyplot as plt
import dwave.inspector

In [56]:
Nfacil = 14
Npos = 81
factory_length = 9
factory_breadth = 9
positions = []
facilities = [i for i in range(Nfacil)]
positions = [i for i in range(Npos)]
np.set_printoptions(linewidth=1000, suppress=True,threshold= sys.maxsize)
flow = pd.read_csv("csv_files/flowmatrixdqm.csv", header = None)
flow = (flow.to_numpy()/3)**3
flow = np.round(flow)
flow = np.triu(flow)
flow = np.array(flow)
print(flow,"\n\n")


distance = rectdistancematrix(factory_length,factory_breadth)
# distance = np.round(distance,decimals=3)
distance = np.triu(np.array(distance))
distance = (np.array(distance))#**3)/5
distance = np.round(distance,decimals=3)
print(distance)
print(np.shape(distance))

print("Facilities = ", facilities)
print("Positions = ",positions)

facility_size_noempty = [10,4,7,4,2,1,14,2,14,3,1,3,2]
emptyspace = [Npos - np.sum(facility_size_noempty)]
facility_size = emptyspace + facility_size_noempty

[[     0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.]
 [     0. 296296.      8.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.]
 [     0.      0. 296296.      8.      8.      8.      1.      1.      2.      0.      2.      2.      1.      2.]
 [     0.      0.      0. 296296.      2.      5.      1.      0.      0.      0.      0.      0.     -2.      0.]
 [     0.      0.      0.      0. 296296.      1.      5.      0.      2.      0.      0.      0.     -2.      0.]
 [     0.      0.      0.      0.      0. 296296.      2.      5.      1.      0.      0.      0.     -2.      0.]
 [     0.      0.      0.      0.      0.      0. 296296.      1.      5.      0.      1.      0.     -2.      0.]
 [     0.      0.      0.      0.      0.      0.      0. 296296.      2.      5.      1.      0.      0.      0.]
 [     0.      0.      0.      0.      0.      0.      0.      0. 296296.      5

#Building a Variable for each Position

In [57]:
labels = []
for p in positions:
    labels.append(f'P{p}')
print(np.array(labels))

['P0' 'P1' 'P2' 'P3' 'P4' 'P5' 'P6' 'P7' 'P8' 'P9' 'P10' 'P11' 'P12' 'P13' 'P14' 'P15' 'P16' 'P17' 'P18' 'P19' 'P20' 'P21' 'P22' 'P23' 'P24' 'P25' 'P26' 'P27' 'P28' 'P29' 'P30' 'P31' 'P32' 'P33' 'P34' 'P35' 'P36' 'P37' 'P38' 'P39' 'P40' 'P41' 'P42' 'P43' 'P44' 'P45' 'P46' 'P47' 'P48' 'P49' 'P50' 'P51' 'P52' 'P53' 'P54' 'P55' 'P56' 'P57' 'P58' 'P59' 'P60' 'P61' 'P62' 'P63' 'P64' 'P65' 'P66' 'P67' 'P68' 'P69' 'P70' 'P71' 'P72' 'P73' 'P74' 'P75' 'P76' 'P77' 'P78' 'P79' 'P80']


#Initialise BQM

In [58]:
dqm = DiscreteQuadraticModel()
for p in positions:
    dqm.add_variable(Nfacil, label=labels[p])
# print(dqm)
print(dqm.variables)

Variables(['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'P38', 'P39', 'P40', 'P41', 'P42', 'P43', 'P44', 'P45', 'P46', 'P47', 'P48', 'P49', 'P50', 'P51', 'P52', 'P53', 'P54', 'P55', 'P56', 'P57', 'P58', 'P59', 'P60', 'P61', 'P62', 'P63', 'P64', 'P65', 'P66', 'P67', 'P68', 'P69', 'P70', 'P71', 'P72', 'P73', 'P74', 'P75', 'P76', 'P77', 'P78', 'P79', 'P80'])


#Objective function

In [59]:
for j in range(len(positions)):
    for jj in range(j+1,len(positions)):
        for i in range(Nfacil):
            for ii in range(Nfacil):
                dqm.set_quadratic_case(labels[j],i,labels[jj],ii,distance[j][jj]*flow[i][ii]/16900)

#constraint 1: only 1 machine is placed per position

In [60]:
# for p in positions:
#     for f in facilities:
#         dqm.add_linear_equality_constraint(
            
#         )
for f in facilities:
    c1 = [(labels[p],f,1) for p in positions]
    dqm.add_linear_equality_constraint(
        c1,
        constant=facilities[f],
        lagrange_multiplier=1
    )

#Constraint 2: only 1 position is chosen per facility

In [61]:
# for f in facilities:
#     c2 = [(x[f][p],1) for p in positions]
#     bqm.add_linear_equality_constraint(
#         c2,
#         constant=-1*facility_size[f],
#         lagrange_multiplier=300,
#         # label = "c2_facil_" + str(f)
#     )

Removing 0 bias variables and couplers from BQM

In [62]:
# new_bqm = BinaryQuadraticModel(bqm.linear, {interaction: bias for interaction, bias in bqm.quadratic.items() if bias}, bqm.offset, bqm.vartype)
file = open("txt_files/dqm.txt", "w")
file.write(str(dqm))
file.close()

# file2 = open("txt_files/new_bqm.txt", "w")
# file2.write(str(new_bqm))
# file2.close()

#running the solver

In [63]:
sampler = LeapHybridDQMSampler()
# numreads = 200 #number of samples for simulated annealer
timelimit = 30 #time limit for hybrid sampler
sampleset = sampler.sample_dqm(dqm,time_limit=timelimit)




In [66]:
print(sampleset.first.sample)
print(sampleset)

{'P0': 0, 'P1': 0, 'P2': 0, 'P3': 6, 'P4': 0, 'P5': 11, 'P6': 0, 'P7': 0, 'P8': 0, 'P9': 0, 'P10': 10, 'P11': 0, 'P12': 6, 'P13': 6, 'P14': 11, 'P15': 11, 'P16': 0, 'P17': 0, 'P18': 0, 'P19': 10, 'P20': 10, 'P21': 13, 'P22': 13, 'P23': 5, 'P24': 5, 'P25': 0, 'P26': 8, 'P27': 4, 'P28': 4, 'P29': 0, 'P30': 0, 'P31': 13, 'P32': 0, 'P33': 5, 'P34': 8, 'P35': 8, 'P36': 4, 'P37': 4, 'P38': 0, 'P39': 0, 'P40': 9, 'P41': 0, 'P42': 0, 'P43': 0, 'P44': 0, 'P45': 0, 'P46': 3, 'P47': 3, 'P48': 0, 'P49': 9, 'P50': 9, 'P51': 7, 'P52': 7, 'P53': 0, 'P54': 0, 'P55': 3, 'P56': 3, 'P57': 0, 'P58': 0, 'P59': 0, 'P60': 7, 'P61': 0, 'P62': 12, 'P63': 2, 'P64': 2, 'P65': 1, 'P66': 1, 'P67': 0, 'P68': 0, 'P69': 0, 'P70': 12, 'P71': 12, 'P72': 2, 'P73': 2, 'P74': 1, 'P75': 1, 'P76': 0, 'P77': 0, 'P78': 0, 'P79': 0, 'P80': 0}
   P0 P1 P2 P3 P4 P5 P6 P7 P8 P9 P10 P11 P12 P13 ... P80      energy num_oc.
1   0  0  0  6  0 11  0  0  0  0  10   0   6   6 ...   0 3991.569662       1
3   0  0  0  6  0 11  0  0  0  0 

Printing Output Solutions

In [65]:
t3 = time.time()

### Extracting best run from all samplesets
energies = [sampleset.first.energy for sampleset in samplesets]
print(energies)
sampleset = samplesets[energies.index(np.min(energies))]
print(sampleset.first.sample)

# dwave.inspector.show_bqm_sampleset(bqm=new_bqm,sampleset=sampleset,sampler=sampler,embedding_context=None,warnings=None,params=None)

### Post Processing the input matrix to convert to a more readable cartesian layout
printout = []
for f in facilities:
    printouttemp = []
    for p in positions:
        label = f'F{f}P{p}'
        value = sampleset.first.sample[label]
        printouttemp.append(value) 
    printout.append(printouttemp)

layout = np.zeros((matrL,matrL))
ctr = 1
for i in printout:
    for j in range(len(i)):
        if i[j] == 1:
            q = int(j/len(layout))
            r = j%len(layout)
            layout[q][r] = ctr
    ctr+=1

### Plotting the Layout using Heatmap
fig, ax = plt.subplots(figsize=(9, 10))
sns.heatmap(layout,annot = layout, vmin = 0, vmax = len(facilities)).set(title = "Final Layout")

### Check to ensure all facilities are assigned correct sizes
correct_size = True
printout = np.array(printout)
for i in range(len(printout)):
    true_size = facility_size[i]
    calc_size = np.sum(printout[i])
    if true_size == calc_size:
        print("Facility ", i+1, "\t correct size of", true_size, "\n")
    else:
        print("Facility ", i+1, "\t wrong size.", "True Size = ", true_size, "Calc size = ", calc_size,"\n")
        correct_size = False
if correct_size == True:
    print("all facilities Correct Size")

### Check to ensure same position is not assigned to two facilities
ctr1 = 0
for i in range(len(printout.T)):
    if np.sum(printout.T[i]) >1:
        print("Position ", i+1, " = Overlapped")
    else:
        ctr1+=1

if ctr1 == 81:
    print("No Overlaps")

fig, ax = plt.subplots(figsize=(17, 9))
sns.heatmap(printout,annot = printout, vmax= 1, vmin=0 ).set(title = "Final Matrix")

t4 = time.time()
print(f'postprocess finished in {t4-t3} seconds') 

NameError: name 'samplesets' is not defined